In [2]:
# | default_exp session

# session

> A group of trials designated by a .mat file.

In [173]:
# | export
from pathlib import Path

import pandas as pd
import plotly.express as px
import scipy
import xarray as xr
from datatree import DataTree


class Session:
    """A group of consecutive trials."""

    def __init__(self, matfile_path: Path):
        self.matfile_path = matfile_path
        self.start_datetime = pd.to_datetime(
            pd.DataFrame.from_records(
                [mat_files.stem.split("_")[1:] for mat_files in mat_files],
                columns=["day", "month", "year", "hour", "minute"],
            ),
        )

    sub_arrays = (
        "EyeJoy",
        "onlineEye",
        "onlineLickForce",
        "samplesBlinkLogical",
        "img1",
        "imgfeedback",
        "spikes",
        "sptimes",
    )

    sparse_arrays = (
        "targetAcquisitionBegin",
        "targetAcquisitionFirst",
        "FEEDbacktime",
        "timeoutcome",
        "outcomedel",
        "ITIeventtime",
    )

    session_columns = (
        "targetacquisitionthreshold",
        "ITI_dur",
    )

    def pds_data(self):
        return scipy.io.loadmat(
            self.matfile_path,
            squeeze_me=True,
        )["PDS"]

    def column_measures(self):
        return self.pds_data().dtype.names

    def data_array(self, measure) -> xr.DataArray:
        _pds_data = self.pds_data()
        return pd.Series(_pds_data[measure].item(), name=measure).to_xarray()

    @property
    def dataset(self) -> xr.Dataset:
        homogenous_arrays = (
            "targAmp",
            "fixreq",
            "windowchosen",
            "blinkthreshold",
            "blinkTimeAllowed",
            "trueCS_dur",
            "rewardduration",
        )
        return xr.Dataset(
            {measure: self.data_array(measure) for measure in homogenous_arrays},
        )

    @property
    def dataframe(self) -> pd.DataFrame:
        return self.dataset.to_dataframe()

    @property
    def datatree(self) -> DataTree:
        return DataTree(name=self.matfile_path.stem, data=self.dataset)

    def eyejoy(self):
        eyejoy_data = pd.DataFrame.from_records(
            self.pds_data()["EyeJoy"].item(),
        ).rename(
            columns={0: "x", 1: "y", 4: "t"},
        )
        eyejoy_data.index.name = "trial"
        trial_t = eyejoy_data[["x", "y", "t"]].explode(column=["x", "y", "t"])
        trial_t["dt"] = pd.to_timedelta(trial_t["t"], unit="s")
        return trial_t

    def plot(self):
        return px.line(
            pd.DataFrame({"trial": [], "proportion_looking": []}),
            x="trial",
            y="proportion_looking",
            title=self.matfile_path.stem,
            template="plotly_white",
        )

In [174]:
from pathlib import Path

data_dir = Path("/workspaces/vns/data/BFINAC_VNS/")
mat_files = data_dir.glob("*.mat")

In [175]:
next_session = Session(next(mat_files))

In [176]:
next_session.dataframe

,targAmp,fixreq,windowchosen,blinkthreshold,blinkTimeAllowed,trueCS_dur,rewardduration
index,,,,,,,
0,10,1,0,-4.8,0.5,3,0.36192
1,10,1,0,-4.8,0.5,3,0.36192
2,10,1,0,-4.8,0.5,3,0.36192
3,10,1,0,-4.8,0.5,3,0.36192
4,10,1,0,-4.8,0.5,3,0.36192
...,...,...,...,...,...,...,...
67,10,1,0,-4.8,0.5,3,0.36192
68,10,1,0,-4.8,0.5,3,0.36192
69,10,1,0,-4.8,0.5,3,0.36192


In [178]:
from ydata_profiling import ProfileReport

profile = ProfileReport(next_session.dataframe, title=next_session.matfile_path.stem)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
profile.to_widgets()

In [95]:
pd.Series(["fixOverlap"].item(), name="fixOverlap")

AttributeError: 'list' object has no attribute 'item'

In [57]:
next_session.eyejoy()

,x,y,t,dt
trial,,,,
0,0.502319,0.214539,2660.990019,0 days 00:44:20.990018780
0,0.493469,0.213318,2660.991019,0 days 00:44:20.991018780
0,0.490723,0.212097,2660.992019,0 days 00:44:20.992018780
0,0.483704,0.209045,2660.993019,0 days 00:44:20.993018780
0,0.483704,0.204773,2660.994019,0 days 00:44:20.994018780
...,...,...,...,...
71,0.0,0.0,0.0,0 days 00:00:00
71,0.0,0.0,0.0,0 days 00:00:00
71,0.0,0.0,0.0,0 days 00:00:00


In [58]:
session_eyejoy = _

In [66]:
good_session_eyejoy = session_eyejoy.set_index("dt", append=True).drop(columns=["t"])
good_session_eyejoy

x         y
trial dt                                           
0     0 days 00:44:20.990018780  0.502319  0.214539
      0 days 00:44:20.991018780  0.493469  0.213318
      0 days 00:44:20.992018780  0.490723  0.212097
      0 days 00:44:20.993018780  0.483704  0.209045
      0 days 00:44:20.994018780  0.483704  0.204773
...                                   ...       ...
71    0 days 00:00:00                 0.0       0.0
      0 days 00:00:00                 0.0       0.0
      0 days 00:00:00                 0.0       0.0
      0 days 00:00:00                 0.0       0.0
      0 days 00:00:00                 0.0       0.0

[720000 rows x 2 columns]

In [69]:
good_session_eyejoy.xs(0).reset_index()

,dt,x,y
0,0 days 00:44:20.990018780,0.502319,0.214539
1,0 days 00:44:20.991018780,0.493469,0.213318
2,0 days 00:44:20.992018780,0.490723,0.212097
3,0 days 00:44:20.993018780,0.483704,0.209045
4,0 days 00:44:20.994018780,0.483704,0.204773
...,...,...,...
9995,0 days 00:00:00,0.0,0.0
9996,0 days 00:00:00,0.0,0.0
9997,0 days 00:00:00,0.0,0.0
9998,0 days 00:00:00,0.0,0.0


In [68]:
px.line(, x="dt", y=["x", "y"])

In [ ]:
px.line(
    good_session_eyejoy.xs(0).reset_index(),
    x="dt",
    y=["x", "y"],
    line_group="trial",
    template="plotly_white",
)